In [ ]:
from collections import defaultdict, Counter
import pprint
from operator import itemgetter
import re,operator
import bz2, json
from collections import defaultdict
import cmath
WORD_LEN_COEFF = 1
THRESHOLD_COEFF = 0.5
DROP = 1.5
DROP_1 = 2
AFFIX_LEN = 1

In [76]:
svoc = bz2.BZ2File('voc.json.bz2', 'r').read().decode(encoding='utf-8')
voc = json.loads(svoc)
del svoc

In [77]:
words = list(voc.keys())
word_count = sum([voc[k] for k in voc])
average_word_len = sum([len(w)*voc[w] for w in words]) / word_count
len_search = int(average_word_len * WORD_LEN_COEFF) #это максимальная разрешенная длина аффикса
print("{} словоформ, {} словоупотреблений, средняя длина слова {} ".format(len(words), word_count, average_word_len))

386233 словоформ, 11859267 словоупотреблений, средняя длина слова 7.1053179762290535 


In [85]:
# подсчет безусловных вероятностей букв
# trie, prob = build_trie_and_prob(voc)
prob = json.load(open("prob.json", encoding="utf-8"))
strie = bz2.BZ2File('trie.json.bz2', 'r').read().decode(encoding='utf-8')
trie = json.loads(strie)
del strie
print("Безусловные вероятности первых 10 букв:\n========================\n",
sorted([(letter,nv) for letter,nv in prob.items()], key=itemgetter(1), reverse=True)[:10])

Безусловные вероятности первых 10 букв:
 [('а', 0.13174638100795355), ('т', 0.07463523242460413), ('н', 0.07183703410321932), ('р', 0.06989005476760542), ('э', 0.06769593508904286), ('и', 0.06497202721408583), ('ы', 0.05657750345483212), ('л', 0.052736912856701096), ('у', 0.048080219156342266), ('о', 0.04801633649290444)]


In [94]:
letters = list(prob.keys())
cond_prob = defaultdict(lambda: 0) #словарь для условных вероятностей
total = defaultdict(lambda: 0)

for word,n in voc.items():#для слова в словаре
        positions = range(-min(len_search, len(word) - 2), 0) # from -7 to 0
        for i in positions:
            cond_prob[(i, word[i])] += n
            total[i] += n # dictionary with prob of char words?

for posChar in cond_prob: #получаем из частот вероятности
        i = posChar[0]
        cond_prob[posChar] /= total[i]
cond_prob

defaultdict(<function __main__.<lambda>>,
            {(-7, 'а'): 0.10984867258690777,
             (-7, 'б'): 0.03237548886108653,
             (-7, 'в'): 0.0034969091918060136,
             (-7, 'г'): 0.012524240720055028,
             (-7, 'д'): 0.026213640352372475,
             (-7, 'е'): 0.017250482808964074,
             (-7, 'ж'): 0.0009517231144666211,
             (-7, 'з'): 0.00270562103944093,
             (-7, 'и'): 0.06383048212711548,
             (-7, 'й'): 0.016326262830857717,
             (-7, 'к'): 0.02338540116332537,
             (-7, 'л'): 0.09235495891575761,
             (-7, 'м'): 0.02007786252499992,
             (-7, 'н'): 0.04784772200975731,
             (-7, 'о'): 0.05104181021059667,
             (-7, 'п'): 0.023812559249920713,
             (-7, 'р'): 0.07725545056296915,
             (-7, 'с'): 0.03281353360582448,
             (-7, 'т'): 0.0791437127687035,
             (-7, 'у'): 0.04925038196413075,
             (-7, 'ф'): 0.0008723651064271106,
   

In [100]:
# информанты - это буквы с макс значением КФ в каждой позиции
max_cond = defaultdict(lambda: 0.0)
maxlet = ['']*8
    #для каждой позиции ищем букву с наибольшим значением условной вероятности,
for posChar in cond_prob:#цикл по позициям букв в условной вероятности
        aff_len = posChar[0]
        if cond_prob[posChar] > max_cond[aff_len]:
            max_cond[aff_len] = cond_prob[posChar]
            maxlet[-aff_len] = posChar[1]

print("Наиболее частые буквы по позициям:\n============================\n", maxlet[-1:0:-1],"\n")

print("Максимальные вероятности по позициям:\n============================\n", max_cond,"\n")
    #порог медиального разбиения - половина условной вероятности , буквы с УВ не меньше порога - верхнее подмножеств
cond_prob_sup = {}
for posChar in cond_prob:
            i = posChar[0]
            if cond_prob[posChar] > THRESHOLD_COEFF * max_cond[i]:
                cond_prob_sup[posChar] = cond_prob[posChar]

Наиболее частые буквы по позициям:
 ['а', 'а', 'т', 'а', 'т', 'а', 'н'] 

Максимальные вероятности по позициям:
 defaultdict(<function <lambda> at 0x00000294381D89D8>, {-6: 0.13725284548556266, -5: 0.10984460793132571, -4: 0.13300193926114215, -3: 0.11746170593037474, -2: 0.21185955687204952, -1: 0.22671356984582827, -7: 0.10984867258690777}) 



In [107]:
# КФ = условная вер по данной позиции / безусл вероятность
cf = {}
for posChar in cond_prob_sup:
        char = posChar[1]
        cf[posChar] = cond_prob_sup[posChar] / prob[char]

print("КФ для верхних подмножества:\n====================\n");
for aff_len in set(map(itemgetter(0), cf.keys())):
        print(aff_len, "**")
        for k,v in cf.items():
            if k[0] == aff_len:
                print(k[1], "{:.4f}".format(v), end="  ")
        print("")

КФ для верхних подмножества:

-1 **
н 3.1559  р 2.0092  а 1.1595  
-7 **
ы 0.9773  р 1.1054  а 0.8338  л 1.7512  т 1.0604  и 0.9824  
-6 **
р 0.9975  а 1.0418  т 0.9701  э 1.0297  л 1.6040  
-5 **
а 0.7812  н 0.8144  т 1.4718  и 0.8949  р 1.1912  л 1.9490  
-4 **
а 1.0095  ы 1.4735  т 1.2438  и 1.2629  э 1.0359  л 1.7136  
-3 **
н 1.4117  л 1.3875  т 1.5738  а 0.8120  р 1.3286  
-2 **
а 1.6081  э 1.7332  


In [121]:
informants = []
for aff_len in range(-len_search, 0):
        kmax = max({k for k in cf if k[0] == aff_len}, key=lambda k: cf[k])
        informants.append((kmax[1], aff_len, cf[kmax]))

informants.sort(key = itemgetter(2), reverse=True)
informants

[('н', -1, 3.15594279017803),
 ('л', -5, 1.94903926924671),
 ('л', -7, 1.751239386475054),
 ('э', -2, 1.7331937693949762),
 ('л', -4, 1.7135874901772652),
 ('л', -6, 1.604008680102341),
 ('т', -3, 1.573810412515745)]

In [122]:
#подсчитываем частоты букв и строим дерево оконочаний
prob = defaultdict(lambda: 0)
trie = {'n':0}
for w,n in voc.items(): #для каждого слова в списке
        word = w[::-1]  # переворачиваем слово, читаем слово с конца
        current_dict = trie
        trie['n'] += n
        for letter in word:  # для буквы в слове
            prob[letter]+=n
            current_dict = current_dict.setdefault(letter, {'n': 0}) #получить значение из словаря по ключу.
                                                                     #Автоматически добавляет элемент словаря, если он отсутствует.
            current_dict['n']+=n
        current_dict['#'] = n

total = sum([n for n in prob.values()])#84263863
for k,v in prob.items():""
        prob[k] = v/total

In [123]:
def extend_right(voc, char, pos, cf):
    if pos == -1:#если информант в последней позиции, то расширять некуда
        return char #возвращаем информант как аффикс
    d = defaultdict(int)
    for w,n in voc.items():#для буквы и частоты в словаре
        if w[pos:pos+1]==char: #если буква в позиции равна нашей, то посчитаем это окончание
            d[w[pos+1:]]+=n
    return char+max(d.keys(), key=lambda end: d[end]) #прибавляем к информанту самое частое окончание

In [151]:
for pos in range (-7,0):
    print(extend_right(voc, char, pos,cf))

ларыгар
лларын
ларын
ллар
лар
ла
л


In [153]:
def extend_left(affix, trie, len_search):
    #расширяем аффикс влево используя trie
   # import ipdb; ipdb.set_trace()
    current_dict = trie
    for ch in affix[::-1]:
        current_dict = current_dict[ch]
    aff_len = len(affix)

In [159]:
informants[0][0]

'н'

In [160]:
print(extend_left(informants[0][0],trie,len_search))

None


In [ ]:
pprint(trie['р']['а']['г']['ы']['р']['а']['л'])

In [179]:
from pprint import pprint
pprint(trie['р']['а']['л']['n'])

186545


In [181]:
pprint(trie['р']['а']['г']['ы']['р']['а']['л'])

{'n': 23661,
 'а': {'#': 1,
       'n': 6653,
       'а': {'n': 579,
             'б': {'n': 5, 'а': {'n': 5, 'л': {'#': 5, 'n': 5}}},
             'д': {'n': 13,
                   'н': {'n': 13, 'ы': {'n': 13, 'м': {'#': 13, 'n': 13}}}},
             'й': {'n': 1,
                   'ы': {'n': 1,
                         'р': {'n': 1,
                               'ы': {'n': 1,
                                     'т': {'n': 1,
                                           'а': {'n': 1,
                                                 'м': {'#': 1, 'n': 1}}}}}}},
             'к': {'n': 2, 'ы': {'n': 2, 'ы': {'n': 2, 'б': {'#': 2, 'n': 2}}}},
             'л': {'n': 373, 'а': {'n': 373, 'с': {'#': 373, 'n': 373}}},
             'н': {'n': 110, 'а': {'n': 110, 'с': {'#': 110, 'n': 110}}},
             'р': {'n': 1, 'а': {'n': 1, 'б': {'#': 1, 'n': 1}}},
             'с': {'#': 4, 'n': 4},
             'т': {'n': 3, 'у': {'#': 3, 'n': 3}},
             'х': {'#': 66, 'n': 66},
          

                                                 'а': {'n': 56,
                                                       'п': {'n': 56,
                                                             'с': {'#': 56,
                                                                   'n': 56}}}}}}},
                         'п': {'n': 35,
                               'м': {'n': 35,
                                     'и': {'n': 35,
                                           'л': {'n': 35,
                                                 'о': {'#': 35, 'n': 35}}}}},
                         'с': {'n': 2,
                               'р': {'n': 2,
                                     'е': {'n': 2,
                                           'в': {'n': 2,
                                                 'и': {'n': 2,
                                                       'н': {'n': 2,
                                                             'у': {'#': 2,
                                  

                   'м': {'n': 5,
                         'о': {'n': 5,
                               'н': {'n': 5,
                                     'о': {'n': 5,
                                           'к': {'n': 5,
                                                 'э': {'#': 5, 'n': 5}}}}}},
                   'н': {'n': 28,
                         'и': {'n': 25,
                               'л': {'n': 25,
                                     'к': {'#': 8,
                                           'n': 25,
                                           'и': {'n': 17,
                                                 'л': {'n': 17,
                                                       'о': {'n': 17,
                                                             'п': {'#': 17,
                                                                   'n': 17}}}}}}},
                         'о': {'n': 2, 'р': {'n': 2, 'х': {'#': 2, 'n': 2}}},
                         'х': {'n': 1, 'е': {'

                   'е': {'n': 1,
                         'т': {'n': 1, 'п': {'n': 1, 'а': {'#': 1, 'n': 1}}}},
                   'о': {'n': 3,
                         'т': {'#': 1,
                               'n': 3,
                               'р': {'n': 2,
                                     'а': {'n': 1, 'к': {'#': 1, 'n': 1}},
                                     'о': {'n': 1, 'ф': {'#': 1, 'n': 1}}}}}},
             'ш': {'n': 1, 'у': {'n': 1, 'п': {'#': 1, 'n': 1}}},
             'ы': {'n': 11,
                   'з': {'n': 6, 'у': {'n': 6, 'м': {'#': 6, 'n': 6}}},
                   'л': {'n': 1,
                         'ы': {'n': 1,
                               'б': {'n': 1,
                                     'а': {'n': 1,
                                           'а': {'n': 1,
                                                 'ь': {'n': 1,
                                                       'д': {'#': 1,
                                                       

                   'п': {'n': 7,
                         'п': {'n': 7,
                               'ы': {'n': 7,
                                     'ы': {'n': 7, 'с': {'#': 7, 'n': 7}}}}},
                   'ҥ': {'n': 3, 'а': {'n': 3, 'т': {'#': 3, 'n': 3}}}},
             'в': {'n': 1, 'е': {'n': 1, 'л': {'n': 1, 'п': {'#': 1, 'n': 1}}}},
             'д': {'n': 1,
                   'е': {'n': 1,
                         'ф': {'n': 1, 'а': {'n': 1, 'к': {'#': 1, 'n': 1}}}}},
             'е': {'n': 7,
                   'м': {'n': 2, 'а': {'n': 2, 'к': {'#': 2, 'n': 2}}},
                   'п': {'n': 2, 'о': {'#': 2, 'n': 2}},
                   'ф': {'n': 1,
                         'с': {'n': 1,
                               'о': {'n': 1,
                                     'м': {'n': 1,
                                           'т': {'n': 1,
                                                 'а': {'#': 1, 'n': 1}}}}}},
                   'ь': {'n': 2,
                    

...

                                     'о': {'n': 20,
                                           'л': {'n': 1, 'о': {'#': 1, 'n': 1}},
                                           'ҥ': {'n': 19,
                                                 'о': {'#': 19, 'n': 19}}}},
                               'с': {'n': 1,
                                     'х': {'n': 1, 'о': {'#': 1, 'n': 1}}},
                               'т': {'n': 2,
                                     'с': {'n': 2,
                                           'о': {'n': 2,
                                                 'х': {'#': 2, 'n': 2}}}},
                               'ь': {'n': 6,
                                     'н': {'n': 6,
                                           'н': {'n': 6,
                                                 'о': {'n': 6,
                                                       'о': {'#': 6,
                                                             'n': 6}}}}}}}}},
       'ь': {'n': 23

                               'һ': {'n': 5,
                                     'а': {'n': 4,
                                           'л': {'n': 4,
                                                 'ы': {'n': 4,
                                                       'р': {'n': 3,
                                                             'а': {'n': 3,
                                                                   'т': {'n': 3,
                                                                         'у': {'#': 3,
                                                                               'n': 3}}}},
                                                       'ы': {'n': 1,
                                                             'т': {'n': 1,
                                                                   'а': {'#': 1,
                                                                         'n': 1}}}}}},
                                     'ы': {'n': 1,
                  

                         'а': {'n': 9,
                               'а': {'n': 2,
                                     'һ': {'n': 2,
                                           'ы': {'n': 2,
                                                 'б': {'#': 2, 'n': 2}}}},
                               'д': {'n': 1,
                                     'р': {'n': 1,
                                           'а': {'n': 1,
                                                 'х': {'#': 1, 'n': 1}}}},
                               'т': {'n': 6, 'у': {'#': 6, 'n': 6}}},
                         'ы': {'n': 18,
                               'с': {'#': 16, 'n': 16},
                               'ы': {'n': 2, 'т': {'#': 2, 'n': 2}}}},
                   'х': {'n': 52,
                         'а': {'n': 52,
                               'р': {'n': 5,
                                     'а': {'#': 3, 'n': 3},
                                     'ы': {'n': 2, 'б': {'#': 2, 'n': 2}}},
             

                   'е': {'n': 2,
                         'л': {'n': 2,
                               'в': {'n': 2,
                                     'а': {'n': 2,
                                           'р': {'n': 2,
                                                 'п': {'n': 2,
                                                       'у': {'#': 2,
                                                             'n': 2}}}}}}},
                   'и': {'n': 7, 'л': {'#': 7, 'n': 7}},
                   'о': {'n': 4,
                         'л': {'n': 3, 'о': {'n': 3, 'к': {'#': 3, 'n': 3}}},
                         'м': {'n': 1,
                               'е': {'n': 1,
                                     'р': {'n': 1,
                                           'е': {'n': 1,
                                                 'ц': {'#': 1, 'n': 1}}}}}},
                   'р': {'n': 1,
                         'е': {'n': 1,
                               'б': {'n': 1,
             

                                                                                     'n': 1}}}}}}}}}},
                         'е': {'n': 36,
                               'в': {'n': 3,
                                     'б': {'n': 1,
                                           'у': {'n': 1,
                                                 'с': {'#': 1, 'n': 1}}},
                                     'н': {'n': 2,
                                           'о': {'n': 2,
                                                 'к': {'#': 2, 'n': 2}}}},
                               'р': {'n': 33,
                                     'е': {'n': 33,
                                           'ф': {'n': 33,
                                                 'н': {'n': 33,
                                                       'о': {'n': 33,
                                                             'к': {'#': 33,
                                                                   'n': 33}}}}}}}